# RSA

## Trapdoor

$X$ y $Y$ conjuntos finitos, $\mathcal{T}$ esta definido sobre $(X, Y)$, $\mathcal{T} = (G, F, I)$. 

- $G$ es el generador de llaves (probabilista).
- $F$ es un algoritmo determinista.
  $$
  y \leftarrow F(P_k, x) \\ 
  y \in Y \\
  x \in X
  $$
- $I$ es un algorimto determinista.
  $$
  x \leftarrow I(S_k, y) <''
  x \in X
  y \in Y
  $$

$$
\forall(P_k, S_k) \xleftarrow{R} G, \forall x in X \\
I(s_k, F(p_k, x)) = x
$$

In [4]:
# Imports
import random 
from typing import Tuple

## Esquemas trapdoor de una vía

$\mathcal{T} = (G, F, I)$ definido sobre $(X, Y)$

**Juego adversarial**

Retador 
$$
(p_k, s_k) \xleftarrow{R} G(parms) \\
x \xleftarrow{R} X \\ 
y \leftarrow F(p_k, x)
$$

Atacente (recibe $p_k, y$)
$$
\hat{x} \in X
$$

$P[x = \hat{x}]$ es bajita, por lo tanto es seguro. 

$X, Y = \mathbf{F}_q$, con $q$ siendo un primo muy grande. si $q = 2018 bits$ entonces 

$$
P[x = \hat{x}] = \cfrac{1}{2018}
$$

**Protocolo de intercambio de llaves**

![Trapddor key exchange](../img/trapdoor-key-exchange.png)

## Trapdoor RSA

In [5]:
def miller_rabin(n: int, k: int = 10) -> bool:
    """Miller-Rabbin primality test

    Args:
        n (int): un número impar para ser testado
        k (int, optional): Un parametro que determina la exactitud del test. Defaults to 10.

    Returns:
        bool: es primo o no 
    """
            
    assert(n > 2 and n % 2 != 0), "n > 2 debe ser impar"
    
    # s, d tal que n = 2**s * d + 1, d impart
    # a random entre 2 y n - 1
    d: int = n - 1
    s: int = 0 
    while d % 2 == 0: 
        d = d // 2
        s += 1 
    assert(n - 1 == 2**s*d), ":p"

    # Ciclo. Repetir k veces
    for _ in range(k): 
        # n is alway a probable prime to base 1 and n - 1
        a = random.randrange(2, n - 1)

        x = pow(a, d, n) # Fuertemente primo en base a

        for _ in range(s): 
            y = pow(x, 2, n)

            if y == 1 and x != 1 and x != n - 1: 
                # Nontrivial squere root of 1 modulo n 
                return False # composite  
            x = y 

        if y != 1:
            return False # Composite 
        
    return True # Probably prime  



def gcd(x: int, y: int) -> int: 
    """Algoritmos de eculides""" 

    b, a = (x, y) if x < y else (y, x) 

    while b > 0: 
        r = a % b 
        a, b = b, r

    return a 


def egcd(x: int, y: int) -> Tuple[int, int, int]:
    """Algoritmo de euclides extendido

    Args:
        x (int)
        y (int)

    Returns:
        (r: int, s:int, t:int) tal que s*x + t*y = r = mdc(x, y)
    """
    r2, r1 = (y, x) if x < y else (x, y)

    s1, s2 = 1, 0
    t1, t2 = 0, 1

    while r2 != 0: 
        q = r1 // r2
        r = r1 % r2 

        r1, s1, t1, r2, s2, t2 = r2, s2, t2, r, s1 - s2*q, t1 - t2*q

    return r1, s1, t1

def modinv(a, b):

    # El pequeño teorema de ferman 
    # Si mcd(a, p) = 1
    # a ^ p - 1 = 1 mod p     
    r, s, _ = egcd(a, b)

    if r != 1:
        raise Exception("Inverse doesn't exist")
    
    return (s % b + b) % b


In [11]:
# Trapddor RSA
class TrapdoorRSA:

    @staticmethod
    def random_prime(l: int, e: int):
        while True:
            p = random.randint(2**(l-1), 2**l - 1)
            # Aseguramos que el número sea impar y sea probablemente primo
            if p % 2 != 0 and miller_rabin(p) and egcd(e, p - 1)[0] == 1:
                return p 
    
    @staticmethod
    def G(p: int, q: int, e: int):
        if e <= 0 and e % 2 == 0:
            raise Exception("e debe ser impar y mayor que cero") 

        if not miller_rabin(p):
            raise Exception("p debe ser primo")
        
        if not miller_rabin(q):
            raise Exception("q debe ser primo")
        
        if egcd(e, p - 1)[0] != 1:
            raise Exception("mdc(e, p - 1) es != 1")
        
        if egcd(e, q - 1)[0] != 1:
            raise Exception("mdc(e, q - 1) es != 1")

        n: int = p * q
        d: int = modinv(e, (p - 1)*(q - 1))

        pk = n, e
        sk = n, d

        return pk, sk
    
    @staticmethod
    def F(pk: int, x: int) -> int: 
        """
        Args:
            pk (int)
            x (int)

        Returns:
            int: y
        """

        n, e = pk
        y = pow(x, e, n)
        return y 
    
    @staticmethod
    def I(sk: int, y: int):
        """
        Args:
            sk (int)
            y (int)

        Returns:
            int: x
        """

        n, d = sk
        x = pow(y, d, n)
        return x 


In [24]:
l = 256
e = 65537

p = TrapdoorRSA.random_prime(l, e)
q = TrapdoorRSA.random_prime(l, e)

try: 
    _pk, _sk = TrapdoorRSA.G(p, q, e)
except Exception as e:
    _pk, _sk = (None, None)

assert(_pk != _sk)

In [4]:
test_cases = [
    (
        114030925945987047454712058480378457694475076087566564082811397486658167069467,
        72305573982007090223813135222531881286173129243195813373713694744350434780059,
        65537,
        (8245071552224338300537003635426687212479215960962094576793151376589388680113921989513721262258972021275639491468017962553755943684958960945352744719358553, 65537),
        (8245071552224338300537003635426687212479215960962094576793151376589388680113921989513721262258972021275639491468017962553755943684958960945352744719358553, 2851939011815821977430662609083228900005971384394307367766665373085506690716668042757063637258977406086354583786474005202625241238686127200732664541370709),
    ),

    (
        74513326052196972144380441489126594354203891516565134471028241794054717830649,
        84657346913278274413505006223762154906112200988230242487155995709316749657383,
        1053817,
        None,
        None,
    ),

    (
        6745630109897752649212284012319794552035468229870850552523133098743562519652302204553243315827167098050726615999876033795795862736271665898635001054131159,
        10396138619899800531822876711522315325976172835848417485190426883950724655114730773784683633371011941039216448661204839830372306391904653501225428229390097,
        3041,
        None,
        None,
    ),

    (
        16543553855020670041,
        12409606627703002801,
        2047,
        (205298995565026068861347840658719784841, 2047),
        (205298995565026068861347840658719784841, 130681285403629197698392900962053216383),
    ),

    (
        8156444034013357564641964320779767538079550565543233315295470092031746928431002415913770792499507001333151222118792133606471072802663911235149821570773509,
        10106465288908653016239805774539274878605644742536567766729512929003597636320858068605772737631224876876776474250433636497028572034270282636892657677373603,
        65537,
        (82432818510682067028683634870556374459975133203419927820978433143562377293900915638405879954409973322731794933176977767782188057285222550141779970070947237889821909621145526476813875569367913470580733717432361385527981285701088242217159669696074085050304055062277301978380615361722316271006574425384288282927, 65537),
        (82432818510682067028683634870556374459975133203419927820978433143562377293900915638405879954409973322731794933176977767782188057285222550141779970070947237889821909621145526476813875569367913470580733717432361385527981285701088242217159669696074085050304055062277301978380615361722316271006574425384288282927, 37472550300902393165975599280766826493607872932790431201345644143201708109005540056752490587023847982526292546946129997677143393848381070446067242448596358806856886689055469416780510264700813284559906626734685364135609743787807617448251614711454437753552081698116104621294419478996784479032058675122556048129),
    ),
]

for i, (p, q, e, pk, sk) in enumerate(test_cases):
    try: 
        _pk, _sk = TrapdoorRSA.G(p, q, e)
    except Exception as e:
        _pk, _sk = (None, None)
    
    assert(_pk == pk), f"pk {i} dosen't match"
    assert(_sk == sk), f"sk {i} dosen't match"

In [5]:
p = 114030925945987047454712058480378457694475076087566564082811397486658167069467
q = 72305573982007090223813135222531881286173129243195813373713694744350434780059
e = 65537

test_cases = [
    (5240904144301058065298374552949913011079721627307828052585483319516544499048121193693452669238649316952808494988567831621085863400843090283704825918324294, 3692579224434883396131185942374335054528197003435161919595510143358658632023200349009072668599628045517250451479325530899576732698234104006459897125614735),
    (5557292131253177968145806283929509410548113867543546375807984248748651630703539142451903051054308063016107974851048692139866635220333620084806649267836950, 3154225082777057646648186062044244151888622320228229613082129643104645195695975565240175142803487252553277175915520210235875650423904111801021978407358584),
    (7887094967805731084177926274714823804842619698510163104231786471458363778381403436378113039215864079605057420831667836649721668630031856726102512182813957, 3456845496503217690157127523866644207859679345661895585985114270755497333370224284402500319002134438132209628879636896694800912858206819915245060740672458),
    (6593395304645743161421591912438729029897432755320740851469611705426181250852209086416985312676849033783746850395153969410859925288990256336480299811077687, 699936825609555011957440904541516233182071222801462588263148562425577264028512652076132537654900065473011354922793330084845277754170717517090417747953327),
    (7449298975215934430765473142814723312039705780050662168746312032821517485801560840584875914275290777666976164518645912893979978580575971497315635260013491, 4747377676122787401760826995560019180665177814097056589825655014005934280877169134902737147137399105304725476122666742176523344870999700461664135355827149)
]

pk, sk = TrapdoorRSA.G(p, q, e)

for i, (x, y) in enumerate(test_cases):
    assert(TrapdoorRSA.F(pk, x) == y)
    assert(TrapdoorRSA.I(sk, y) == x)


## RSA OAEP

In [358]:
import hashlib

class RSA_OAEP: 

    @staticmethod
    def H(x: int, h: int):
        x_bytes = x.to_bytes((x.bit_length() + 7) // 8, 'big') 
        hashed = hashlib.sha256(x_bytes).digest() 
        return int.from_bytes(hashed[:h//8], 'big') % (1 << h)


    @staticmethod
    def W(x: int, t: int, h: int):
        x_bytes = x.to_bytes((x.bit_length() + 7) // 8, 'big')  #
        hashed = hashlib.sha256(x_bytes).digest()
        return int.from_bytes(hashed[:(t - h - 8)//8], 'big') % (1 << (t - h - 8))

    @staticmethod
    def padding(m: int, t: int, h: int, d: int = None): 
        r = random.randrange(1 << (h - 1), 1 << h)
        
        if d is not None:
            d = d % (2 ** h)
        
        d_len = len((bin(d)[2:]))
        m_len = len((bin(m)[2:]))
        
        if d is not None:
            pad = bin(d)[2:] + "0" * (t - h - 8 - m_len -d_len - 2) + "01"
        else:
            pad = "0" * (t - h - 8 - m_len - 2) + "01"

        z = int(pad + bin(m)[2:], 2)
        

        # z' = W(r) XOR z
        # r' = H(z') XOR r
        z_ = RSA_OAEP.W(r, t, h) ^ z 
        r_ = RSA_OAEP.H(z_, h) ^ r  

        print(len(bin(z)[2:]), t - h - 8)
        print(len(bin(z_)[2:]), t - h - 8)

        print("\n", len(bin(r)[2:]), h)
        print(len(bin(r_)[2:]), h)

        x = (0 << t - h) | (r_ << (len(bin(z_)[2:]) - 8)) | z_
        
        print(len(bin(x)) + 8, t, bin(x))
        return x 
    
    
    @staticmethod
    def random_prime(l: int, e: int):
        return TrapdoorRSA.random_prime(l, e)
    
    @staticmethod
    def G(p: int, q: int, e: int):
        return TrapdoorRSA.G(p, q, e)
    
    

    @staticmethod
    def E(pk: int, m: int):
        m_ = RSA_OAEP.padding(m)

        pass

    @staticmethod
    def D(sk: int, y: int):
        # TODO 
        pass

In [374]:
t = 512  # Longitud total del mensaje x
h = 256  # Longitud de los valores aleatorios r y r'
m = 123456789  # El mensaje m (en este caso, solo un número de ejemplo)
d = 987654321  # Relleno opcional d

# Aplicar el relleno al mensaje m
RSA_OAEP.padding(m, t, h, d)

248 248
247 248

 256 256
255 256
504 512 0b10001101110111000011001010111101000101001111101000101111001000010011100111111111111001011001000010011101100001111000011100010000101010101111100110010011010101010010010001011101010001011101111011000100111111011101011000101100110101111000101110101111010011110010110010000001111100011101100110100100111100000000000011010000010001010011100010110101001101101010001111001010011011011110001111110011000101001110001011000010000010010111111001110100001011010000011101000000001001001000001010111011100100


28342509769134490831419787315505760411252472317046528023963135797672282480513395725447233043258240478725354198524628063428944426385384067492655181540